# Deploy the knowledge base assistant

Now that we have a basic understanding of the knowledge base and LLM, it is time to combine them skillfully and create a visual interface that is not only more convenient for operation, but also easier to share with others.

Streamlit is a quick and easy way to demonstrate machine learning models through a friendly web interface directly in Python. In this course, we will learn how to use it to build user interfaces for generative AI applications. After building a machine learning model, if you want to build a demo for others to see, maybe to get feedback and drive improvements to the system, or just because you think the system is cool, you want to demonstrate it: Streamlit allows you to quickly achieve this goal through a Python interface program without writing any front-end, web page, or JavaScript code.

## 1. Introduction to Streamlit

`Streamlit` is an open source Python library for quickly creating data applications. It is designed to enable data scientists to easily transform data analysis and machine learning models into interactive web applications without having to deeply understand web development. Unlike conventional web frameworks such as Flask/Django, it does not require you to write any client-side code (HTML/CSS/JS). You only need to write ordinary Python modules to create beautiful and highly interactive interfaces in a short time, thereby quickly generating data analysis or machine learning results; on the other hand, unlike those tools that can only be generated by dragging and dropping, you still have full control over the code.

Streamlit provides a set of simple and powerful basic modules for building data applications:

- st.write(): This is one of the most basic modules used to present text, images, tables, and other content in the application.

- st.title(), st.header(), st.subheader(): These modules are used to add titles, subtitles, and grouped titles to organize the layout of the application.

- st.text(), st.markdown(): Used to add text content, supporting Markdown syntax.

- st.image():Used to add images to the application.

- st.dataframe(): used to render Pandas data frames.

- st.table(): used to render simple data tables.

- st.pyplot(), st.altair_chart(), st.plotly_chart(): used to render charts drawn by Matplotlib, Altair or Plotly.

- st.selectbox(), st.multiselect(), st.slider(), st.text_input(): used to add interactive widgets that allow users to select, enter or slide in the application.

- st.button(), st.checkbox(), st.radio(): used to add buttons, checkboxes and radio buttons to trigger specific actions.

These basic modules make it easy to build interactive data applications through Streamlit, and they can be combined and customized as needed when used. For more information, please check the [official documentation](https://docs.streamlit.io/get-started)

## 2. Build the application

First, create a new Python file and save it streamlit_app.py in the root of your working directory.

1. Import necessary Python libraries.

In [3]:
import streamlit as st
from langchain_openai import ChatOpenAI

2. Create the application title `st.title`

In [ ]:
st.title('🦜🔗 动手学大模型应用开发')

3. Add a text input box for users to enter their OpenAI API key

In [ ]:
openai_api_key = st.sidebar.text_input('OpenAI API Key', type='password')

4. Define a function that uses the user's key to authenticate to the OpenAI API, send a prompt, and get the AI-generated response. The function accepts the user's prompt as a parameter and uses `st.info` to display the AI-generated response in a blue box

In [ ]:
def generate_response(input_text):
    llm = ChatOpenAI(temperature=0.7, openai_api_key=openai_api_key)
    st.info(llm(input_text))

5. Finally, use `st.form()` to create a text box (st.text_area()) for the user to enter input. When the user clicks `Submit`, `generate-response()` will call the function with the user's input as a parameter

In [ ]:
with st.form('my_form'):
    text = st.text_area('Enter text:', 'What are the three key pieces of advice for learning how to code?')
    submitted = st.form_submit_button('Submit')
    if not openai_api_key.startswith('sk-'):
        st.warning('Please enter your OpenAI API key!', icon='⚠')
    if submitted and openai_api_key.startswith('sk-'):
        generate_response(text)

6. Save the current file `streamlit_app.py`!

7. Return to your computer's terminal to run the application

In [ ]:
streamlit run streamlit_app.py

The results are shown below: 

![](../../figures/streamlit_app.png)

However, only a single round of conversation is currently possible. We make some modifications to the above. By using `st.session_state` to store the conversation history, the context of the entire conversation can be retained when the user interacts with the application.

![](../../figures/streamlit_app2.png)

The specific code is as follows:

In [ ]:
# Streamlit API
def main():
    st.title('🦜🔗 动手学大模型应用开发')
    openai_api_key = st.sidebar.text_input('OpenAI API Key', type='password')

# Used to track conversation history
    if 'messages' not in st.session_state:
        st.session_state.messages = []

    messages = st.container(height=300)
    if prompt := st.chat_input("Say something"):
# Add user input to the conversation history
        st.session_state.messages.append({"role": "user", "text": prompt})

# Call the respond function to get the answer
        answer = generate_response(prompt, openai_api_key)
# Check if the answer is None
        if answer is not None:
# Add LLM's answer to the conversation history
            st.session_state.messages.append({"role": "assistant", "text": answer})

# Display the entire conversation history
        for message in st.session_state.messages:
            if message["role"] == "user":
                messages.chat_message("user").write(message["text"])
            elif message["role"] == "assistant":
                messages.chat_message("assistant").write(message["text"])   

## 3. Add search questions and answers

First, encapsulate the code in the section `2. Build a search question and answer chain`:
- The get_vectordb function returns the vector knowledge base persisted in the C3 section
- The get_chat_qa_chain function returns the result of calling a search question and answer chain with historical records
- The get_qa_chain function returns the result of calling a search question and answer chain without historical records

In [ ]:
def get_vectordb():
# Define Embeddings
    embedding = ZhipuAIEmbeddings()
# Vector database persistence path
    persist_directory = '../C3 搭建知识库/data_base/vector_db/chroma'
# Load the database
    vectordb = Chroma(
        persist_directory=persist_directory,  # 允许我们将persist_directory目录保存到磁盘上
        embedding_function=embedding
    )
    return vectordb

#Question and answer chain with history
def get_chat_qa_chain(question:str,openai_api_key:str):
    vectordb = get_vectordb()
    llm = ChatOpenAI(model_name = "gpt-3.5-turbo", temperature = 0,openai_api_key = openai_api_key)
    memory = ConversationBufferMemory(
        memory_key="chat_history",  # 与 prompt 的输入变量保持一致。
        return_messages=True  # 将以消息列表的形式返回聊天记录，而不是单个字符串
    )
    retriever=vectordb.as_retriever()
    qa = ConversationalRetrievalChain.from_llm(
        llm,
        retriever=retriever,
        memory=memory
    )
    result = qa({"question": question})
    return result['answer']

#Q&A chain without history
def get_qa_chain(question:str,openai_api_key:str):
    vectordb = get_vectordb()
    llm = ChatOpenAI(model_name = "gpt-3.5-turbo", temperature = 0,openai_api_key = openai_api_key)
    template = """使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答
        案。最多使用三句话。尽量使答案简明扼要。总是在回答的最后说“谢谢你的提问！”。
        {context}
        问题: {question}
        """
    QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context","question"],
                                 template=template)
    qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=vectordb.as_retriever(),
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt":QA_CHAIN_PROMPT})
    result = qa_chain({"query": question})
    return result["result"]

Then, add a radio button component `st.radio` to select the mode for question and answer:
- None: Normal mode without search and answer
- qa_chain: Search and answer mode without history
- chat_qa_chain: Search and answer mode with history

In [ ]:
selected_method = st.radio(
        "你想选择哪种模式进行对话？",
        ["None", "qa_chain", "chat_qa_chain"],
        captions = ["不使用检索问答的普通模式", "不带历史记录的检索问答模式", "带历史记录的检索问答模式"])

The final effect is as follows: 

![](../../figures/streamlit_app3.png)

Enter the page, first enter OPEN_API_KEY (default), then click the radio button to select the question and answer mode, and finally enter your question in the input box and press Enter! 

> For the complete code, refer to [streamlit_app.py](./streamlit_app.py)

## 4. Deploy your app 

To deploy your app to Streamlit Cloud, follow these steps: 

1. Create a GitHub repository for your app. Your repository should contain two files: 

your-repository/ 
├── streamlit_app.py 
└── requirements.txt 

2. Go to [Streamlit Community Cloud](http://share.streamlit.io/), click the `New app` button in your workspace, and specify the repository, branch, and master file path. Optionally, you can customize the URL of your app by selecting a custom subdomain

3. Click the `Deploy!` button 

Your app will now be deployed to Streamlit Community Cloud and accessible from all over the world! 🌎

Our project deployment is basically completed. We have simplified it for demonstration purposes. There are still many areas that can be further optimized. We look forward to learners making various modifications!

Optimization direction:
- Add the function of uploading local documents and establishing vector database in the interface
- Add buttons for selecting multiple LLM and embedding methods
- Add buttons for modifying parameters
- More......